In [1]:
# import dependencies for converting csv to dataframe and computing distance
import pandas as pd
from math import radians, sin, cos, sqrt, atan2

# convert Wpi Data.csv to a dataframe for easy manipulation
ndf = pd.read_csv(r'C:\Users\23470\OneDrive\Desktop\10Alytics\capstone_project\project3\Wpi Data.csv')
ndf = pd.DataFrame(ndf)

# obtaining relevant colunms from the Wpi Data
column_of_interest = ['Wpi_country_code', 'Main_port_name', 'Latitude_degrees', 'Latitude_minutes', 'Latitude_hemisphere', 'Longitude_degrees', 'Longitude_minutes', 'Longitude_hemisphere']
col_considered = ndf[column_of_interest]
col_considered.head()



,Wpi_country_code,Main_port_name,Latitude_degrees,Latitude_minutes,Latitude_hemisphere,Longitude_degrees,Longitude_minutes,Longitude_hemisphere
0,IS,KEFLAVIK,64,0,N,22,33,W
1,IS,STRAUMSVIK,64,3,N,22,3,W
2,IS,HAFNARFJORDUR,64,4,N,21,57,W
3,IS,SKERJAFJORDUR,64,9,N,22,1,W
4,IS,REYKJAVIK,64,9,N,21,56,W


In [ ]:
# converting latitude and longitude hemisphere to lag or lead, N = 1, S = -1, E = 1, W = -1
# case latitude
lat_hemis = col_considered['Latitude_hemisphere']
lat_hemis_list = lat_hemis.values.tolist()

lat_lag = []
for i in lat_hemis_list:
    if i == 'N':
        lat_lag.append(1)
    else: 
        lat_lag.append(-1)

# create a new colunm called 'latitude_lag' and assign lat_lag to it
col_considered['latitude_lag'] = lat_lag
        
# case longitude
lon_hemis = col_considered['Longitude_hemisphere']
lon_hemis_list = lon_hemis.values.tolist()

lon_lag = []
for i in lon_hemis_list:
    if i == 'E':
        lon_lag.append(1)
    else: 
        lon_lag.append(-1)
        
# create a new colunm called 'longitude_lag' and assign lon_lag to it
col_considered['longitude_lag'] = lon_lag

In [ ]:
# convert latitude to degrees in decimal and assigning it a new col called Latitude_decimal_degree
col_considered['Latitude_decimal_degree'] = (col_considered['Latitude_degrees'] + (col_considered['Latitude_minutes']/60))*col_considered['latitude_lag']

# convert longitude to degrees in decimal and assigning it a new col called Longitude_decimal_degree
col_considered['Longitude_decimal_degree'] = (col_considered['Longitude_degrees'] + (col_considered['Longitude_minutes']/60))*col_considered['longitude_lag']


In [6]:
# convert Latitude_decimal_degree to list
lat_list = (col_considered['Latitude_decimal_degree']).values.tolist()

# convert Longitude_decimal_degree to list
lon_list = (col_considered['Longitude_decimal_degree']).values.tolist()



In [ ]:
# define a function for computing distance
def calculate_distance(lat_a, lon_a, lat_series, lon_series):
    # Earth's radius in kilometers
    radius = 6371.0

    # Convert point A's coordinates from degrees to radians
    lat_a_rad = radians(lat_a)
    lon_a_rad = radians(lon_a)

    distances = []
    for lat_b, lon_b in zip(lat_series, lon_series):
        # Convert point B's coordinates from degrees to radians
        lat_b_rad = radians(lat_b)
        lon_b_rad = radians(lon_b)

        # Difference between the latitudes and longitudes
        d_lat = lat_b_rad - lat_a_rad
        d_lon = lon_b_rad - lon_a_rad

        # Haversine formula
        a = sin(d_lat / 2)**2 + cos(lat_a_rad) * cos(lat_b_rad) * sin(d_lon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = round((radius * c)*1000, 2)

        distances.append(distance)

    return distances

# Coordinates of point A
lat_a = 1.283333 #col_considered.loc[col_considered.Main_port_name == 'JURONG ISLAND', 'Latitude_decimal_degree']
lon_a = 103.733333 #col_considered.loc[col_considered.Main_port_name == 'JURONG ISLAND', 'Longitude_decimal_degree']

# Coordinates of the series of points


distances = calculate_distance(lat_a, lon_a, lat_list, lon_list)
print(distances)

In [ ]:
col_considered['Distance_meters_Jurong'] = distances
col_considered.to_csv('distance_jurong_meters.csv', index = False)

In [12]:
DF = pd.read_csv('distance_jurong_meters.csv')
DF = pd.DataFrame(DF)
DF.head()

,Wpi_country_code,Main_port_name,Latitude_degrees,Latitude_minutes,Latitude_hemisphere,Longitude_degrees,Longitude_minutes,Longitude_hemisphere,latitude_lag,longitude_lag,Latitude_decimal_degree,Longitude_decimal_degree,Distance_meters_Jurong
0,IS,KEFLAVIK,64,0,N,22,33,W,1,-1,64.000000,-22.550000,11546562.62
1,IS,STRAUMSVIK,64,3,N,22,3,W,1,-1,64.050000,-22.050000,11523214.27
2,IS,HAFNARFJORDUR,64,4,N,21,57,W,1,-1,64.066667,-21.950000,11518132.88
3,IS,SKERJAFJORDUR,64,9,N,22,1,W,1,-1,64.150000,-22.016667,11515737.68
4,IS,REYKJAVIK,64,9,N,21,56,W,1,-1,64.150000,-21.933333,11512363.95


In [18]:
col = ['Main_port_name', 'Distance_meters_Jurong']
col_con = DF[col]
col_con.to_csv('col_con_jurong.csv', index = False)
col_con.head()

,Main_port_name,Distance_meters_Jurong
0,KEFLAVIK,11546562.62
1,STRAUMSVIK,11523214.27
2,HAFNARFJORDUR,11518132.88
3,SKERJAFJORDUR,11515737.68
4,REYKJAVIK,11512363.95


In [2]:
# define a connect to postgres sever
import psycopg2
def get_db_connection(host, database, username, password):
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=username,
        password=password
    )
    
    return conn

# initialize connection function parameters
host = "localhost"
database = "World_Port_index"
username = "postgres"
password = "kw#s#k00"

# establish a connect
con = get_db_connection(host, database, username, password)
# create cursor
curr = con.cursor()

In [ ]:
create_statement = 'CREATE TABLE distance_jurong(Main_port_name VARCHAR, Distance_meters_Jurong NUMERIC)'
curr.execute(create_statement)
con.commit()

In [22]:
with open('col_con_jurong.csv', 'r') as f:
    next(f)
    curr.copy_from(f, 'distance_jurong', sep=',')
    
# commit connection   
con.commit()

curr.close()
con.close()

In [3]:
sql = """select * from distance_jurong
where Main_port_name != 'JURONG ISLAND'
order by Distance_meters_Jurong asc
LIMIT 5 """

pd.read_sql(sql, con)

,main_port_name,distance_meters_jurong
0,PULAU BUKOM,6681.48
1,PULAU SEBAROK,11865.49
2,KEPPEL - (EAST SINGAPORE),12969.52
3,TANJUNG PELEPAS,21686.64
4,PULAU SAMBU,22618.32
